<a href="https://colab.research.google.com/github/dindaalutfi/NLP_Analisis-Sentimen/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd # Pandas juga dapat membaca file dari berbagai format seperti .txt, .csv, .tsv, dan lainnya
import re # ekspresi reguler adalah urutan karakter khusus yang membantu Anda mencocokkan atau menemukan string atau kumpulan string lain
import string 
import nltk # libray python untuk bekerja dengan permodelan teks
nltk.download('punkt')
nltk.download('stopwords')

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from wordcloud import WordCloud

from string import punctuation

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df = pd.read_csv("gdrive/MyDrive/NLP_Dataset Sentiment Analysis/ulasan_MyXL.csv")
df.head()

,userName,score,at,Ulasan,Sentimen Widi,Sentimen Nanda,"Sentimen Dra. UMI KOLISOH, M.Pd",Sentimen,Keterangan:
0,Deden Herdiana,4,2022-02-11 8:42:00,Harusnya dikasih bintang 4 bilang terimakasih....,1,1,1,-1,1. negatif
1,Herry Ghunawan,1,2022-02-11 7:53:00,Tolong dong masalah jaringan hampir setiap har...,1,1,1,-1,2. positif
2,Miyuki,5,2022-02-11 5:50:00,"Saya mau komen lagi,miminnnn kenapa akhir akhi...",1,1,1,-1,3. netral
3,Wildan Saat Almaarif,2,2022-02-11 5:33:00,Bonus kouta tiktok 13gb tidak diaktifkan/tidak...,1,1,1,-1,NaN
4,putri purwantiy,3,2022-02-11 3:10:00,"Sejauh ini bagusÂ² aja sih apk nya, tpi udh bb...",3,3,3,0,NaN


In [ ]:
df = df[['Ulasan', 'score']]
print(df.iloc[:10, :])

                                              Ulasan  score
0  Harusnya dikasih bintang 4 bilang terimakasih....      4
1  Tolong dong masalah jaringan hampir setiap har...      1
2  Saya mau komen lagi,miminnnn kenapa akhir akhi...      5
3  Bonus kouta tiktok 13gb tidak diaktifkan/tidak...      2
4  Sejauh ini bagusÂ² aja sih apk nya, tpi udh bb...      3
5  bagus murah murah harganya tapi kalo bisa tamb...      5
6  Ini aplikasi aneh, di sms suruh beli paket via...      1
7  Padahal cuma ngaktifin bonus, kuota utama ilan...      1
8  jringan xl di tmpt ku blm lancar, kalau mau on...      4
9  Asu lagi enak2 main game, gangguan mulu di akh...      1


In [ ]:
#Labelling data
#Asumsi rating 4,5 = Positif
#dan rating 1,2,3 = Negatif
print("Please wait while creating label...")
df.loc[df['score'] > 3, 'label'] = 0
df.loc[df['score'] <= 3, 'label'] = 1
df.to_csv('cleaned_data.csv', header=True, index=False, encoding='utf-8')
print(df)

Please wait while creating label...
                                                Ulasan  score  label
0    Harusnya dikasih bintang 4 bilang terimakasih....      4    0.0
1    Tolong dong masalah jaringan hampir setiap har...      1    1.0
2    Saya mau komen lagi,miminnnn kenapa akhir akhi...      5    0.0
3    Bonus kouta tiktok 13gb tidak diaktifkan/tidak...      2    1.0
4    Sejauh ini bagusÂ² aja sih apk nya, tpi udh bb...      3    1.0
..                                                 ...    ...    ...
995  Xl sekarang beda sama dulu, sekarang xl borosn...      1    1.0
996  Sebagai Pelanggan Setia XL Bertahun tahun , Ke...      2    1.0
997  Suka boros kalo pake sekali beli mahal bgt kal...      1    1.0
998  Kenapa jaringan XL leletnya minta ampun? Jarin...      1    1.0
999  Sedang terjadi kesalahan Maaf, untuk saat ini ...      1    1.0

[1000 rows x 3 columns]


In [ ]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas
df['Ulasan'] = df['Ulasan'].str.lower()


print('Case Folding Result : \n')
print(df['Ulasan'].head(5))
print('\n\n\n')

Case Folding Result : 

0    harusnya dikasih bintang 4 bilang terimakasih....
1    tolong dong masalah jaringan hampir setiap har...
2    saya mau komen lagi,miminnnn kenapa akhir akhi...
3    bonus kouta tiktok 13gb tidak diaktifkan/tidak...
4    sejauh ini bagusâ² aja sih apk nya, tpi udh bb...
Name: Ulasan, dtype: object






In [ ]:
# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

# ------ Tokenizing ---------

def remove_tweet_special(Ulasan):
    # remove tab, new line, ans back slice
    Ulasan = Ulasan.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    Ulasan = Ulasan.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    Ulasan = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", Ulasan).split())
    # remove incomplete URL
    return Ulasan.replace("http://", " ").replace("https://", " ")
                
df['Ulasan'] = df['Ulasan'].apply(remove_tweet_special)

#remove number
def remove_number(Ulasan):
    return  re.sub(r"\d+", "", Ulasan)

df['Ulasan'] = df['Ulasan'].apply(remove_number)

#remove punctuation
def remove_punctuation(Ulasan):
    return Ulasan.translate(str.maketrans("","",string.punctuation))

df['Ulasan'] = df['Ulasan'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(Ulasan):
    return Ulasan.strip()

df['Ulasan'] = df['Ulasan'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(Ulasan):
    return re.sub('\s+',' ',Ulasan)

df['Ulasan'] = df['Ulasan'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(Ulasan):
    return re.sub(r"\b[a-zA-Z]\b", "", Ulasan)

df['Ulasan'] = df['Ulasan'].apply(remove_singl_char)

# NLTK word rokenize 
def word_tokenize_wrapper(Ulasan):
    return word_tokenize(Ulasan)

df['text_tokens'] = df['Ulasan'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n') 
print(df['text_tokens'].head())
print('\n\n\n')

Tokenizing Result : 

0    [harusnya, dikasih, bintang, bilang, terimakas...
1    [tolong, dong, masalah, jaringan, hampir, seti...
2    [saya, mau, komen, lagimiminnnn, kenapa, akhir...
3    [bonus, kouta, tiktok, gb, tidak, diaktifkanti...
4    [sejauh, ini, bagus, aja, sih, apk, nya, tpi, ...
Name: text_tokens, dtype: object






In [ ]:
# NLTK calc frequency distribution
def freqDist_wrapper(Ulasan):
    return FreqDist(Ulasan)

df['text_tokens_fdist'] = df['text_tokens'].apply(freqDist_wrapper)

print('Frequency Tokens : \n') 
print(df['text_tokens_fdist'].head().apply(lambda x : x.most_common()))

Frequency Tokens : 

0    [(bintang, 2), (harusnya, 1), (dikasih, 1), (b...
1    [(tolong, 1), (dong, 1), (masalah, 1), (jaring...
2    [(saya, 4), (jaringan, 3), (kek, 3), (kenapa, ...
3    [(kouta, 2), (bonus, 1), (tiktok, 1), (gb, 1),...
4    [(ini, 2), (sinyal, 2), (ya, 2), (kak, 2), (tu...
Name: text_tokens_fdist, dtype: object


In [ ]:
from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])

# ----------------------- add stopword from txt file ------------------------------------
# read txt stopword using pandas
#txt_stopword = pd.read_csv("stopwords.txt", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
#list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))

# ---------------------------------------------------------------------------------------

# convert list to dictionary
list_stopwords = set(list_stopwords)


#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

df['text_tokens_WSW'] = df['text_tokens'].apply(stopwords_removal) 


print(df['text_tokens_WSW'].head())

0    [dikasih, bintang, terimakasih, ngasih, bintan...
1    [tolong, jaringan, leg, parahh, kecewa, costumer]
2    [komen, lagimiminnnn, jaringan, xl, parah, sek...
3    [bonus, kouta, tiktok, gb, diaktifkantidak, sa...
4    [bagus, apk, tpi, udh, bbrp, sinyal, xl, hilan...
Name: text_tokens_WSW, dtype: object


In [ ]:
!pip3 install swifter
!pip3 install PySastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
df.head()

,Ulasan,score,label,text_tokens,text_tokens_fdist,text_tokens_WSW
0,harusnya dikasih bintang bilang terimakasih be...,4,0.0,"[harusnya, dikasih, bintang, bilang, terimakas...","{'harusnya': 1, 'dikasih': 1, 'bintang': 2, 'b...","[dikasih, bintang, terimakasih, ngasih, bintan..."
1,tolong dong masalah jaringan hampir setiap har...,1,1.0,"[tolong, dong, masalah, jaringan, hampir, seti...","{'tolong': 1, 'dong': 1, 'masalah': 1, 'jaring...","[tolong, jaringan, leg, parahh, kecewa, costumer]"
2,saya mau komen lagimiminnnn kenapa akhir akhir...,5,0.0,"[saya, mau, komen, lagimiminnnn, kenapa, akhir...","{'saya': 4, 'mau': 1, 'komen': 1, 'lagimiminnn...","[komen, lagimiminnnn, jaringan, xl, parah, sek..."
3,bonus kouta tiktok gb tidak diaktifkantidak di...,2,1.0,"[bonus, kouta, tiktok, gb, tidak, diaktifkanti...","{'bonus': 1, 'kouta': 2, 'tiktok': 1, 'gb': 1,...","[bonus, kouta, tiktok, gb, diaktifkantidak, sa..."
4,sejauh ini bagus aja sih apk nya tpi udh bbrp ...,3,1.0,"[sejauh, ini, bagus, aja, sih, apk, nya, tpi, ...","{'sejauh': 1, 'ini': 2, 'bagus': 1, 'aja': 1, ...","[bagus, apk, tpi, udh, bbrp, sinyal, xl, hilan..."


In [ ]:
# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter


# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in df['text_tokens_WSW']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

df['text_tokens_stemmed'] = df['text_tokens_WSW'].swifter.apply(get_stemmed_term)
print(df['text_tokens_stemmed'])

4242
------------------------
dikasih : kasih
bintang : bintang
terimakasih : terimakasih
ngasih : ngasih
jaringan : jaring
area : area
lemot : lot
terkadang : terkadang
drop : drop
tolong : tolong
leg : leg
parahh : parahh
kecewa : kecewa
costumer : costumer
komen : komen
lagimiminnnn : lagimiminnnn
xl : xl
parah : parah
sekalii : sekali
pas : pas
searching : searching
google : google
behhh : behhh
loadingnya : loadingnya
kek : kek
nungguin : nungguin
ayam : ayam
jantan : jantan
bertelurpadahal : bertelurpadahal
berasa : asa
habis : habis
fikir : fikir
gini : gin
sekaliiii : sekaliiii
miminku : miminku
tercinta : cinta
lebihhhhhhhhhhh : lebihhhhhhhhhhh
ditingkatkan : tingkat
baiklah : baik
say : say
segitu : segitu
ntar : ntar
dh : dh
ilang : ilang
kesabaran : sabar
jaringannya : jaring
lagii : lagi
bonus : bonus
kouta : kouta
tiktok : tiktok
gb : gb
diaktifkantidak : diaktifkantidak
sayajadi : sayajadi
gbjadi : gbjadi
gbitu : gbitu
merugikan : rugi
bagus : bagus
apk : apk
tpi : tpi
u

Pandas Apply:   0%|          | 0/1000 [00:00<?, ?it/s]

0      [kasih, bintang, terimakasih, ngasih, bintang,...
1        [tolong, jaring, leg, parahh, kecewa, costumer]
2      [komen, lagimiminnnn, jaring, xl, parah, sekal...
3      [bonus, kouta, tiktok, gb, diaktifkantidak, sa...
4      [bagus, apk, tpi, udh, bbrp, sinyal, xl, hilan...
                             ...                        
995    [xl, beda, xl, boros, banget, iya, gb, gk, nya...
996    [langgan, setia, xl, tahun, kecewa, layan, xl,...
997    [suka, boros, pake, beli, mahal, bgt, dapet, b...
998    [jaring, xl, lelet, ampun, jaring, pdahal, pak...
999    [salah, maaf, masuk, layan, myxl, silah, coba,...
Name: text_tokens_stemmed, Length: 1000, dtype: object


In [ ]:
df.to_csv("hasil_processing.csv")

In [ ]:
!pip3 install -U scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np

df2 = pd.read_csv("hasil_processing.csv", usecols=["text_tokens_stemmed"])
df2.columns = ["Ulasan"]

df2.head()

,Ulasan
0,"['kasih', 'bintang', 'terimakasih', 'ngasih', ..."
1,"['tolong', 'jaring', 'leg', 'parahh', 'kecewa'..."
2,"['komen', 'lagimiminnnn', 'jaring', 'xl', 'par..."
3,"['bonus', 'kouta', 'tiktok', 'gb', 'diaktifkan..."
4,"['bagus', 'apk', 'tpi', 'udh', 'bbrp', 'sinyal..."


In [ ]:
#Vectorization
#Scikit-belajar ini CountVectorizer digunakan untuk mengkonversi koleksi dokumen teks ke vektor istilah / jumlah tanda
from sklearn.feature_extraction.text import CountVectorizer

# Untuk membuat Count Vectorizer, kita hanya perlu membuatnya.
# Ada parameter khusus yang dapat kita atur di sini saat membuat vectorizer, tetapi
# untuk contoh paling dasar, ini tidak diperlukan.
cv = CountVectorizer()
# Untuk benar-benar membuat vectorizer, kita hanya perlu memanggil fit pada teks
# data yang ingin kita perbaiki
cv.fit(df['Ulasan'])
# Jika kita benar-benar ingin membuat vektor, kita dapat melakukannya dengan memasukkan teks 
# ke dalam vectorizer untuk mendapatkan hitungan mundur
X = cv.transform(df['Ulasan'])

y = df['label']

In [ ]:
#Build Classifier
#sklearn adalah modul Python yang mengintegrasikan algoritme pembelajaran mesin klasik dalam dunia paket Python ilmiah yang erat 

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.80)

In [ ]:
#Find the best value of C in logistic regression
#Regresi Logistik adalah algoritma klasifikasi Pembelajaran Mesin yang digunakan untuk memprediksi probabilitas variabel dependen kategoris

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print('Accuracy for C=%s: %s'
         % (c, accuracy_score(y_test, lr.predict(X_test))))

Accuracy for C=0.01: 0.76
Accuracy for C=0.05: 0.8
Accuracy for C=0.25: 0.805
Accuracy for C=0.5: 0.81
Accuracy for C=1: 0.81


In [ ]:
#Find the best value of C in support vector
#Mesin vektor pendukung (SVM) adalah sekumpulan metode pembelajaran yang diawasi yang digunakan untuk klasifikasi, regresi dan deteksi pencilan

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    sv = SVC(C=c)
    sv.fit(X_train, y_train)
    print('Accuracy for C=%s: %s'
         % (c, accuracy_score(y_test, sv.predict(X_test))))

Accuracy for C=0.01: 0.76
Accuracy for C=0.05: 0.76
Accuracy for C=0.25: 0.76
Accuracy for C=0.5: 0.76
Accuracy for C=1: 0.785


In [ ]:
#Here I choose C=1 to build the final model for Logistic Regression.
final_model_lr = LogisticRegression(C=1)
final_model_lr.fit(X, y)
print('Final Model Accuracy: %s' %accuracy_score(y_test, final_model_lr.predict(X_test)))
print('Precision: %s' %precision_score(y_test, final_model_lr.predict(X_test)))
print('Recall: %s' %recall_score(y_test, final_model_lr.predict(X_test)))

Final Model Accuracy: 0.99
Precision: 0.993421052631579
Recall: 0.993421052631579


In [ ]:
#Here I choose C=1 to build the final model for Support Vector.
final_model_sv = SVC(C=1)
final_model_sv.fit(X, y)
print('Final Model Accuracy: %s' %accuracy_score(y_test, final_model_sv.predict(X_test)))
print('Precision: %s' %precision_score(y_test, final_model_sv.predict(X_test)))
print('Recall: %s' %recall_score(y_test, final_model_sv.predict(X_test)))

Final Model Accuracy: 0.945
Precision: 0.9325153374233128
Recall: 1.0
